In [51]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
from torchvision import datasets, transforms

In [52]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=0.5, std=0.5),
                              ])

trainset = datasets.MNIST('MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

for images, labels in trainloader:
    print(type(images))
    print(images.shape)
    print(labels.shape)
    break
    
inputs = images.view(images.shape[0], -1)
print(inputs.shape)

<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 784])


In [53]:
n_input = inputs.shape[1]
n_hidden = 256
n_output = len(torch.unique(labels))
print(n_input)

784


In [54]:
def softmax (scores):    
    return torch.div(torch.exp(scores), torch.sum(torch.exp(scores), dim=1).view(-1,1))

In [58]:

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.linear(n_input, n_hidden)
        self.output = nn.linear(n_hidden, n_output)
        
    def forward(self, x):
        x = self.hidden(x)
        x = F.sigmoid(x)
        x = self.output(x)
        x = F.softmax(x)
        
        return x

In [59]:
model = Network()
model

AttributeError: module 'torch.nn' has no attribute 'linear'